In [1]:
# For ML Models
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.losses import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.metrics import *
from tensorflow.keras import regularizers
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import load_img

# For Data Processing
import numpy as np

# For Data Visualization
import matplotlib.pyplot as plt

# Miscellaneous
import os
import random

# Turn off warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [ ]:
TRAIN_SIZE = 384
INFERENCE_SIZE = 224

In [ ]:
main_dir = '/kaggle/input/flickr-image-dataset/flickr30k_images/flickr30k_images/'
all_image_paths = [main_dir+file for file in os.listdir(main_dir) if file.endswith('.jpg')]

print('Total number of images:', len(all_image_paths))

In [ ]:
train_image_paths = all_image_paths[:25000]

remaining_paths = all_image_paths[25000:]

num_remaining = len(remaining_paths)
num_valid = num_remaining // 2
num_test = num_remaining - num_valid  # Acest lucru asigură că toate imaginile rămase sunt utilizate

valid_image_paths = remaining_paths[:num_valid]
test_image_paths = remaining_paths[num_valid:]

In [ ]:
def open_images(paths, size=TRAIN_SIZE):
    '''
    Given an array of paths to images, this function opens those images,
    and returns them as an array of shape (None, Height, Width, Channels)
    '''
    images = []
    for path in paths:
        image = load_img(path, target_size=(size, size, 3))
        image = np.array(image)/255.0 # Normalize image pixel values to be between 0 and 1
        images.append(image)
    return np.array(images)

In [ ]:
def add_noise(images, amount=0.1):
    '''
    Given an array of images [a shape of (None, Height, Width, Channels)],
    this function adds gaussian noise to every channel of the images
    '''
    # Create a matrix with values with a mean of 0 and standard deviation of "amount"
    noise = np.random.normal(0, amount, images.shape[0]*images.shape[1]*images.shape[2]*images.shape[3]).reshape(images.shape)
    # Add noise to images
    noise_img = images+noise
    return noise_img

In [ ]:
image = open_images([train_image_paths[400]])
noise_img = add_noise(image, amount=0.2)

fig = plt.figure(figsize=(10, 5))
# Plot Image
fig.add_subplot(1, 2, 1)
plt.axis('off')
plt.title('Image')
plt.imshow(image[0])
fig.add_subplot(1, 2, 2)
plt.axis('off')
plt.title('Image with noise')
plt.imshow(noise_img[0])

plt.show()

In [ ]:
def datagen(paths, size=TRAIN_SIZE, batch_size=5, is_training=True):
 
    for x in range(0, len(paths), batch_size):
        batch_paths = paths[x:x+batch_size]
        batch_images = open_images(batch_paths, size=size)
        if is_training:
            amount = random.uniform(0, 0.2)  # Amount of noise = random value between 0 and 0.2
            noise_images = add_noise(batch_images, amount=amount)
        else:
            noise_images = batch_images  # No noise added for validation data
        yield noise_images, batch_images

In [ ]:
image = Input(shape=(None,None,3))

# encoder
l1 = Conv2D(64, (3,3), padding='same', activation='relu')(image)     
l2 = Conv2D(64, (3,3), padding='same', activation='relu')(l1)

l3 = MaxPooling2D(padding='same')(l2)
l3 = Dropout(0.3)(l3)
l4 = Conv2D(128, (3,3), padding='same', activation='relu')(l3)
l5 = Conv2D(128, (3,3), padding='same', activation='relu')(l4)

l6 = MaxPooling2D(padding='same')(l5)
l7 = Conv2D(256, (3,3), padding='same', activation='relu',)(l6)

#decoder
l8 = UpSampling2D()(l7)
l9 = Conv2D(128, (3,3), padding='same', activation='relu')(l8)
l10 = Conv2D(128, (3,3), padding='same', activation='relu')(l9)

l11 = add([l5,l10])
l12 = UpSampling2D()(l11)
l13 = Conv2D(64, (3,3), padding='same', activation='relu')(l12)
l14 = Conv2D(64, (3,3), padding='same', activation='relu')(l13)

l15 = add([l14,l2])

decoded = Conv2D(3, (3,3), padding='same', activation='relu')(l15)
model = Model(image, decoded)

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=False, dpi=70)

In [ ]:
# model.compile(optimizer='adam',
#               loss='mean_squared_error',
#               metrics=['mean_squared_error'])

learning_rate = 0.00001  
optimizer = Adam(learning_rate=learning_rate)

model.compile(optimizer=optimizer,
              loss='mean_squared_error')

In [ ]:
from tensorflow.keras.models import load_model

# Înlocuiește acesta cu calea la modelul specific pe care dorești să-l încarci
model_path = '/kaggle/input/model-nou/autoencoder_model_epoch_5.h5'
model = load_model(model_path)


In [ ]:
batch_size=10
steps = int(len(train_image_paths)/batch_size)
valid_steps = len(valid_image_paths) // batch_size
# Liste pentru stocarea istoricului de loss
train_losses = []
val_losses = []
epochs = 5

# Antrenează modelul și acumulează istoricul
for epoch in range(epochs):
    history = model.fit(
        datagen(train_image_paths, size=TRAIN_SIZE, batch_size=batch_size, is_training=True),
        validation_data=datagen(valid_image_paths, size=TRAIN_SIZE, batch_size=batch_size),
        epochs=1,
        steps_per_epoch=steps,
        validation_steps=valid_steps
    )
    train_losses.extend(history.history['loss'])
    val_losses.extend(history.history['val_loss'])

    model.save(f'/kaggle/working/autoencoder_model_epoch_{epoch+6}.h5')


In [ ]:
import matplotlib.pyplot as plt

# Creează o listă cu numărul de epoci pentru axa x
epochs_range = range(1, len(train_losses) + 1)

# Plotare
plt.figure(figsize=(8, 5))
plt.plot(epochs_range, train_losses, label='Training Loss')
plt.plot(epochs_range, val_losses, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
batch_size = 10
steps = int(len(test_image_paths) / batch_size)
loss = model.evaluate(datagen(test_image_paths, size=INFERENCE_SIZE, batch_size=batch_size), steps=steps)
print("Loss on test set:", loss)

In [ ]:
def plot_results(noise_image, reconstructed_image, image):
    w = 15
    h = len(noise_image)*5
    fig = plt.figure(figsize=(w, h))
    columns = 3
    rows = len(noise_image)
    for i in range(1, rows*columns, columns):
        fig.add_subplot(rows, columns, i)
        plt.axis('off')
        plt.title('Image with noise')
        plt.imshow(noise_images[int((i-1)/columns)])
    
        fig.add_subplot(rows, columns, i+1)
        plt.axis('off')
        plt.title('Reconstructed Image')
        plt.imshow(reconstructed[int((i-1)/columns)])
        
        fig.add_subplot(rows, columns, i+2)
        plt.axis('off')
        plt.title('Original Image')
        plt.imshow(images[int((i-1)/columns)])
    
    plt.show()
    
batch_size = 7

paths = random.sample(test_image_paths, batch_size)
images = open_images(paths, size=INFERENCE_SIZE)
# Amount of noise = random value between 0.1 and 0.15
amount = random.uniform(0.1,0.15)
noise_images = add_noise(images, amount=amount)
reconstructed = model.predict(noise_images)

plot_results(noise_images, reconstructed, images)

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array, save_img
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import tensorflow as tf

model = load_model('/kaggle/input/denoise-model/autoencoder_model (5).h5')
TRAIN_SIZE = 384

def denoise_img(image_path, save_path=None):
    """
    This function takes an image path, denoises the image using the trained model,
    saves the denoised image to a specified path, and displays the original and denoised images.
    
    :param image_path: path to the input image
    :param save_path: path to save the denoised image (if None, it won't save)
    """
    # Load and preprocess the image
    original_img = load_img(image_path)
    img = load_img(image_path, target_size=(TRAIN_SIZE, TRAIN_SIZE))  # Resize image for the model
    img_array = img_to_array(img) / 255.0  # Normalize the image array
    img_array = np.expand_dims(img_array, axis=0)  # Expand dims to fit model input

    # Predict the denoised image
    denoised_img_array = model.predict(img_array)[0]
    
    # Rescale the pixels values
    denoised_img_array = np.clip(denoised_img_array, 0, 1)
    
    # Resize back to original dimensions
    denoised_img = tf.image.resize(denoised_img_array, (original_img.size[1], original_img.size[0]))
    denoised_img = np.clip(denoised_img.numpy(), 0, 1)  # Convert to numpy and clip any potential rounding discrepancies

    # Save the denoised image if a save_path is provided
    if save_path:
        save_img(save_path, denoised_img * 255)  # Multiply by 255 to convert back to original scale

    # Display the original and the denoised images
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(original_img)
    plt.title('Original Image')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(denoised_img)
    plt.title('Denoised Image')
    plt.axis('off')

    plt.show()

image_path = '/kaggle/input/poza-caine-pisica/istockphoto-489272417-612x612.jpg'
save_path = '/kaggle/working/denoise.jpg'
denoise_img(image_path, save_path)